In [1]:
import math

y=[[0.3, 0.1], [-0.5, 0.2], [0.8, 0.5], [-0.5, 0.3], [0.1, -0.7], [1.5, -0.4]]
k=6 # 총 시간
s=4 # state 개수
dp=4 # decimal point (소수점 자리 수)

# trellis diagram / 인덱스는 현재 state를 뜻함
trellis=[
  [[0,[-1,-1]],[2,[1,1]]],
  [[0,[1,1]],[2,[-1,-1]]],
  [[1,[1,-1]],[3,[-1,1]]],
  [[1,[-1,1]],[3,[1,-1]]]
]

# 현재 상태로 비트를 전송하는 이전 상태
# ex) state3의 이전 state 2개는 각각 come_in[3][0][0], come_in[3][1][0]
# ex) state come_in[3][0][0]에서 전송하는 비트는 come_in[3][0][1]
come_in=[[[0,0],[1,0]], [[2,0],[3,0]],[[0,1],[1,1]],[[2,1],[3,1]]]

a = [[0 for col in range(s)] for row in range(k+1)]
b = [[0 for col in range(s)] for row in range(k+1)]
r = [[[0 for col in range(s)] for col in range(s)] for row in range(k+1)]


#γ 구하기
for i in range(1, k+1):
    for j in range(s):
        nxt=trellis[j][0][0]
        r[i][j][nxt]=round(math.exp(2.5*((y[i-1][0]*trellis[j][0][1][0])+(y[i-1][1]*trellis[j][0][1][1]))),dp)
        nxt=trellis[j][1][0]
        r[i][j][nxt]=round(math.exp(2.5*((y[i-1][0]*trellis[j][1][1][0])+(y[i-1][1]*trellis[j][1][1][1]))),dp)
    print("r(gamma)")
    print(r)
    print()


#α 구하기
a[0]=[1,0,0,0]
for i in range(1, k+1):
    sum=0
    for j in range(s):
        a[i][j]=a[i-1][come_in[j][0][0]]*r[i][come_in[j][0][0]][j]+a[i-1][come_in[j][1][0]]*r[i][come_in[j][1][0]][j]
        sum=sum+a[i][j]
    for j in range(s):
        a[i][j]=round(a[i][j]/sum, dp)
    print("a(alpha)")
    print(a)
    print()


#β 구하기
b[6]=[1,0,0,0]
for i in range(k-1, -1, -1):
    sum=0
    for j in range(s):
        b[i][j]=b[i+1][trellis[j][0][0]]*r[i+1][j][trellis[j][0][0]]+b[i+1][trellis[j][1][0]]*r[i+1][j][trellis[j][1][0]]
        sum=sum+b[i][j]
    for j in range(s):
        b[i][j]=round(b[i][j]/sum, dp)
    print("b(beta)")
    print(b)
    print()


#decoding
L=[] # L(uk|y)
u=[] # decoding result
binary_bit=[] # decoding result with {0, 1} values

for i in range(1, k+1):
    r0=r1=0 # 비트 0, 1에 대한 P(s',s|y) 합산
    for j in range(s):
        ps=come_in[j][0][0] # 첫 번째 이전 상태
        psb=come_in[j][0][1] # 첫 번째 이전 상태로부터 받은 비트
        if psb==0:
            r0=r0+a[i-1][ps]*r[i][ps][j]*b[i][j]
        else:
            r1=r1+a[i-1][ps]*r[i][ps][j]*b[i][j]
        ps=come_in[j][1][0] # 두 번째 이전 상태
        psb=come_in[j][1][1] # 두 번째 이전 상태로부터 받은 비트
        if psb==0:
            r0=r0+a[i-1][ps]*r[i][ps][j]*b[i][j]
        else:
            r1=r1+a[i-1][ps]*r[i][ps][j]*b[i][j]
    if r1==0:
        L.append(float("-inf"))
    elif r0==0:
        L.append(float("inf"))
    else:
        L.append(round(math.log((r1/(r0+r1))/(r0/(r0+r1))), dp))
for i in L:
    if i>0:
        u.append(1)
        binary_bit.append(1)
    else:
        u.append(-1)
        binary_bit.append(0)



r(gamma)
[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0.3679, 0, 2.7183, 0], [2.7183, 0, 0.3679, 0], [0, 1.6487, 0, 0.6065], [0, 0.6065, 0, 1.6487]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]]

r(gamma)
[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0.3679, 0, 2.7183, 0], [2.7183, 0, 0.3679, 0], [0, 1.6487, 0, 0.6065], [0, 0.6065, 0, 1.6487]], [[2.117, 0, 0.4724, 0], [0.4724, 0, 2.117, 0], [0, 0.1738, 0, 5.7546], [0, 5.7546, 0, 0.1738]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]]

r(gamma)
[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0,

In [2]:
print("Received sequence")
print(y)
print()

print("L(uk|y)")
print(L)
print()

print("u")
print(u)
print()

print("with {0, 1} values")
print(binary_bit)

Received sequence
[[0.3, 0.1], [-0.5, 0.2], [0.8, 0.5], [-0.5, 0.3], [0.1, -0.7], [1.5, -0.4]]

L(uk|y)
[1.7771, 0.238, -1.9732, 5.5863, -inf, -inf]

u
[1, 1, -1, 1, -1, -1]

with {0, 1} values
[1, 1, 0, 1, 0, 0]
